In [1]:
import pandas as pd
import statistics as stats
import csv
import sys

In [16]:
def column_select():
    
    yes_no = ['yes', 'no']
    stat_list = ['mean', 'median', 'sd']
    
    def stat(stat_selection, data, column_title):
        if stat_selection == 'mean':
            print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.mean(data)))
        elif stat_selection == 'median':
            print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.median(data))) 
        elif stat_selection == 'sd':
            print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.stdev(data)))
        while True:
            cont_ = input('Would you like to start over?(yes or no): ')
            if cont_.lower() not in yes_no:
                print('Please input yes or no only')
            elif cont_.lower() == 'no':
                print('')
                print('Thanks, have a nice day!')
                sys.exit(1)
            elif cont_.lower() == 'yes':
                column_select()
                return None
    
    def mean(data, column_title):
        print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.mean(data)))
        while True:
            cont_ = input('Would you like to start over?(yes or no): ')
            if cont_.lower() not in yes_no:
                print('Please input yes or no only')
            elif cont_.lower() == 'no':
                print('')
                print('Thanks, have a nice day!')
                sys.exit(1)
            elif cont_.lower() == 'yes':
                column_select()
                return None
        
    def median(data, column_title):
        print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.median(data))) 
        while True:
            cont_ = input('Would you like to start over?(yes or no): ')
            if cont_.lower() not in yes_no:
                print('Please input yes or no only')
            elif cont_.lower() == 'no':
                print('')
                print('Thanks, have a nice day!')
                sys.exit(1)
            elif cont_.lower() == 'yes':
                column_select()
                return None
        
    def sd(data, column_title):
        print('The mean of' + ' ' + column_title + 'is' + ' ' + str(stats.stdev(data)))
        while True:
            cont_ = input('Would you like to start over?(yes or no): ')
            if cont_.lower() not in yes_no:
                print('Please input yes or no only')
            elif cont_.lower() == 'no':
                print('')
                print('Thanks, have a nice day!')
                sys.exit(1)
            elif cont_.lower() == 'yes':
                column_select()
                return None
    
    #Get column titles
    with open (r'C:/Users/mq814d/Documents/Stuff/more_stuff/test/data/avocado.csv') as file:
        reader = csv.reader(file)
        column_labels = next(reader)
        print('\033[1m' + """Columns""")
        print('\n'.join(column_labels))
    while True:
        try:
            column_name = input('Please input a column name from the list above: ')
            column_name = column_name.lower().title()
            if column_name not in column_labels:
                print('That column does not exist..')
            else:
                while True: 
                    correct_column = input('You have selected' + ' ' + column_name + ' ' + 'is that correct?(yes or no): ')
                    if correct_column.lower() not in yes_no:
                                print('Please input yes or no only...')
                    elif correct_column.lower() == 'no':
                        column_select()
                    elif correct_column.lower() == 'yes':                    
                        df = pd.read_csv(r'C:/Users/mq814d/Documents/Stuff/more_stuff/test/data/avocado.csv', usecols = ['%s' %(column_name)])
                        print('')
                        while True:
                            stat_select = input('Would you like to know the mean, median or standard deviation(sd) of' + ' ' + column_name + '?(mean, median, or sd): ')
                            if stat_select not in stat_list:
                                print('Please input mean, median, or sd only')
                            elif stat_select in stat_list:
                                stat(stat_select, df['%s' %(column_name)], column_name)
        except ValueError: 
            print('That column does not exist...')
            return None

In [17]:
column_select()

Columns
ID
Date
AveragePrice
Total Volume
4046
4225
4770
Total Bags
Small Bags
Large Bags
XLarge Bags
type
year
region


Please input a column name from the list above:  type


That column does not exist..


Please input a column name from the list above:  date
You have selected Date is that correct?(yes or no):  yes


Would you like to know the mean, median or standard deviation(sd) of Date?(mean, median, or sd):  mean


TypeError: can't convert type 'str' to numerator/denominator